In [1]:
import pandas as pd

df = pd.read_csv("existing_relevant_combinations.csv", delimiter=",", keep_default_na=False)

relevant_columns = ["model", "dataset", "attack", "norm", "epsilon", 
          "target", "epe", "px1", "px3", "px5", "cosim", "outlier", "l2_delta12", 
          "l0_delta12", "l_inf_delta12", "epe_orig_preds", "cosim_target", "cosim_orig_preds", 
          "epe_ground_truth_to_negative", "epe_ground_truth_to_zero", "name", "epe_ground_truth", 
          "3dcc_corruption", "model_parameters", "point_matching_method"]

df = df[relevant_columns]
df["attack"] = df["attack"].replace("none", "clean")

# for col in df.columns:
#     print(col)

df

/scratch/ipykernel_106631/2487591171.py:3: DtypeWarning: Columns (294) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("existing_relevant_combinations.csv", delimiter=",", keep_default_na=False)


,model,dataset,attack,norm,epsilon,target,epe,px1,px3,px5,...,epe_orig_preds,cosim_target,cosim_orig_preds,epe_ground_truth_to_negative,epe_ground_truth_to_zero,name,epe_ground_truth,3dcc_corruption,model_parameters,point_matching_method
0,ccmr,kitti-2015,bim,inf,0.0314,negative,68.701383,0.1033126995284465,0.13694128588571403,0.16720623803834314,...,5.8251438620686535,-0.8171277339756489,0.8171277339756489,,,nan,5.501489339619875,,10780884.0,attention
1,ccmr,kitti-2015,bim,inf,0.0314,zero,32.059995,0.1337974854553204,0.1999400941584463,0.2557522329928179,...,5.705682454407215,0.0,0.8340890837460756,,,nan,5.54389090642333,,10780884.0,attention
2,ccmr,kitti-2015,bim,inf,0.0314,nan,4.890594,0.7238333242735826,0.8426503218780271,0.8738035485334694,...,,,,,,nan,,,10780884.0,attention
3,ccmr,kitti-2015,bim,inf,0.0314,nan,11.112749,0.23362163420466459,0.4482095036507235,0.5524682925350498,...,,,,,,nan,,,10780884.0,attention
4,ccmr,kitti-2015,bim,inf,0.0314,nan,4.890594,0.7238333242735826,0.8426503218780271,0.8738035485334694,...,,,,,,nan,,,10780884.0,attention
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5093,videoflow_bof,sintel-final,fgsm,two,0.0,zero,13.014498,,,,...,,,,,,nan,1.09205556154702,,12659389.0,correlation
5094,videoflow_bof,sintel-final,fgsm,two,0.251,nan,1.663153,0.8231735058132412,0.935432811434972,0.9580532020238818,...,,,,,,nan,,,12659389.0,correlation
5095,videoflow_bof,sintel-final,pgd,two,0.251,negative,23.846916,0.16180220729574402,0.3287004796242658,0.4310784616819126,...,2.3471950653378606,-0.800886286211922,0.800886286211922,,,nan,2.9778061236874858,,12659389.0,correlation
5096,videoflow_bof,sintel-final,pgd,two,0.251,zero,10.965081,0.2687617422412109,0.4891782419762571,0.5893780290158426,...,2.2005412022473148,0.0,0.8897423109810846,,,nan,2.857968842612506,,12659389.0,correlation


In [2]:
# Example function to reshape the DataFrame for a single attack
def reshape_for_attack(df, attack_name):
    # Filter rows for the specific attack
    attack_df = df[df['attack'] == attack_name].copy()
    
    # Drop unnecessary columns if needed
    attack_df = attack_df.drop(columns=['attack'], errors='ignore')
    
    # Create a pivot table with a multi-level column index
    pivot = attack_df.pivot_table(
        index=['model', 'dataset'], 
        columns=['norm', 'epsilon', 'target', 'name', '3dcc_corruption'],
        aggfunc='first'
    )
    
    # Flatten the multi-index columns and create unique names
    pivot.columns = [
        f"{attack_name}_{'_'.join(map(str, col))}" for col in pivot.columns
    ]
    
    # Reset the index for merging later
    pivot.reset_index(inplace=True)
    return pivot

# Combine results for all attacks
def transform_dataframe(df, attacks):
    result = pd.DataFrame()
    for attack in attacks:
        attack_pivot = reshape_for_attack(df, attack)
        if result.empty:
            result = attack_pivot
        else:
            result = pd.merge(result, attack_pivot, on=['model', 'dataset'], how='outer')
    return result


attacks = df['attack'].unique()  # Get unique attack names
transformed_df = transform_dataframe(df, attacks)

transformed_df.to_csv('transformed_data.csv', index=False)

transformed_df

,model,dataset,bim_cosim_inf_0.0157_nan_nan_,bim_cosim_inf_0.0314_nan_nan_,bim_cosim_inf_0.0314_negative_nan_,bim_cosim_inf_0.0314_zero_nan_,bim_cosim_two_0.251_nan_nan_,bim_cosim_two_0.251_negative_nan_,bim_cosim_two_0.251_zero_nan_,bim_cosim_orig_preds_inf_0.0157_nan_nan_,...,weather_outlier_nan__negative_nan_,weather_outlier_nan__zero_nan_,weather_point_matching_method_nan__negative_nan_,weather_point_matching_method_nan__zero_nan_,weather_px1_nan__negative_nan_,weather_px1_nan__zero_nan_,weather_px3_nan__negative_nan_,weather_px3_nan__zero_nan_,weather_px5_nan__negative_nan_,weather_px5_nan__zero_nan_
0,ccmr,kitti-2015,0.1804731972562149,0.17941779897548257,,,0.18292504290584474,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ccmr,sintel-clean,0.9011316607553265,0.8835697042309356,,,0.9196476581131576,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ccmr,sintel-final,0.8720889007395328,0.847652535524137,,,0.8864337926592739,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,craft,kitti-2015,0.17133171987719834,0.1667808724590577,,,0.18305703392717987,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,craft,sintel-clean,0.4995292919335885,0.4080966372475901,,,0.8664160683258159,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,starflow,sintel-clean,0.8399587799575785,0.7983058794418584,,,0.9103869874599344,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,starflow,sintel-final,0.7988400316624866,0.7643497439099549,,,0.8949224072992515,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,videoflow_bof,kitti-2015,0.16907726036035456,0.1611451965107699,NaN,NaN,0.1797929968405515,NaN,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,videoflow_bof,sintel-clean,0.4640022494974061,0.38632374172216133,,,0.8715166759826637,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
for col in transformed_df.columns:
    print(col)

model
dataset
bim_cosim_inf_0.0157_nan_nan_
bim_cosim_inf_0.0314_nan_nan_
bim_cosim_inf_0.0314_negative_nan_
bim_cosim_inf_0.0314_zero_nan_
bim_cosim_two_0.251_nan_nan_
bim_cosim_two_0.251_negative_nan_
bim_cosim_two_0.251_zero_nan_
bim_cosim_orig_preds_inf_0.0157_nan_nan_
bim_cosim_orig_preds_inf_0.0314_nan_nan_
bim_cosim_orig_preds_inf_0.0314_negative_nan_
bim_cosim_orig_preds_inf_0.0314_zero_nan_
bim_cosim_orig_preds_two_0.251_nan_nan_
bim_cosim_orig_preds_two_0.251_negative_nan_
bim_cosim_orig_preds_two_0.251_zero_nan_
bim_cosim_target_inf_0.0157_nan_nan_
bim_cosim_target_inf_0.0314_nan_nan_
bim_cosim_target_inf_0.0314_negative_nan_
bim_cosim_target_inf_0.0314_zero_nan_
bim_cosim_target_two_0.251_nan_nan_
bim_cosim_target_two_0.251_negative_nan_
bim_cosim_target_two_0.251_zero_nan_
bim_epe_inf_0.0157_nan_nan_
bim_epe_inf_0.0314_nan_nan_
bim_epe_inf_0.0314_negative_nan_
bim_epe_inf_0.0314_zero_nan_
bim_epe_two_0.251_nan_nan_
bim_epe_two_0.251_negative_nan_
bim_epe_two_0.251_zero_nan